In [139]:
# import autograd functionally
import autograd.numpy as np
from autograd.misc.flatten import flatten_func
from autograd import grad as compute_grad

# import various other libraries
import copy
import matplotlib.pyplot as plt

# this is needed to compensate for %matplotl+ib notebook's tendancy lotted inline
from matplotlib import rcParams
rcParams["figure.autolayout"] = True

In [140]:
# gradient descent function
def gradient_descent(g, w ,alpha, max_its, beta):
    # flatten the input function, create gradient based on flat function
    g_flat, unflatten, w = flatten_func(g, w)
    grad = compute_grad(g_flat)

    # record history
    w_hist = []
    # push the first w
    w_hist.append(unflatten(w))

    # start gradient descent loop
    z = np.zeros(np.shape(w))  # momentum term

    # over the line
    for k in range(max_its):
        # plug in value into func and derivative
        grad_eval = grad(w)
        grad_eval.shape = np.shape(w)

        # take descent step with momentum
        z = beta * z + grad_eval
        w = w -alpha * z

        # record weight update
        w_hist.append(unflatten(w))

    return w_hist

def normalize(data, data_mean, data_std):
    normalized = (data - data_mean)/data_std
    return normalized

def model(x ,w):
    # feature transformations
    f = w[0] + np.dot(x, w[1:])
    return f

def plotting(cost_hist, ylabelName, label):
    figure, axes = plt.subplots(1,1, figsize = (6,5))
    axes.plot(range(len(cost_hist)),cost_hist, label = label, linestyle = "solid")
    axes.set_xlabel("iterations")
    axes.set_ylabel(ylabelName)
    axes.legend()
    plt.show()

In [141]:
csvname = "credit_dataset.csv"
data = np.loadtxt(csvname, delimiter = ',')
data = data.T
x = data[:,:-1]
y = data[:,-1:]

In [142]:
# we do the normalization of these data matrix
x_means = np.mean(x, axis = 0)
x_stds = np.std(x, axis = 0)
# normalize the input data
x_normed = normalize(x, x_means, x_stds)

In [143]:
def sign(x):
    a_1 = np.array(x>=0, dtype = np.int)
    a_2 = np.array(x<0, dtype = np.int)
    return a_1*1 + a_2*(-1)

def softmax_cost(w):
    cost = np.sum(np.log(1+np.exp(-y*model(x_normed,w))))
    return cost/float(np.size(y))

def perceptron_cost(w):
    cost = np.sum(np.maximum(0,-y*model(x_normed,w)))
    return cost/float(np.size(y))


In [144]:
# initialize parameters
alpha = 0.05
max_its = 1000
beta = 0
w_init = np.random.randn(x.shape[1]+1, 1)

In [145]:
# run gradient descent, create cost function history
weight_history = gradient_descent(softmax_cost, w_init, alpha, max_its,beta)
# use MSE to validate the regression quality
cost_history = [softmax_cost(v) for v in weight_history]

In [146]:
best_w = weight_history[-1]

In [147]:
def accuracy(x,y):
    result = np.array(x == y, dtype = int)
    number = np.sum(result)
    return number/np.size(y)

In [148]:
accuracy(sign(model(x_normed,best_w)),y)

0.779

In [149]:
from sklearn.metrics import confusion_matrix
y_pred = sign(model(x_normed,best_w))
y_true = y

In [150]:
y_pred = [y_pred[i][0] for i in range(np.size(y))]

In [151]:
y_true = data[:,-1]

# Here we show the confusion matrix

In [152]:
confusionMatrix = confusion_matrix(y_true, y_pred)
confusionMatrix

array([[148, 152],
       [ 69, 631]])

In [153]:
accuracy = (confusionMatrix[0][0]+confusionMatrix[1][1])/(np.sum(confusionMatrix))

In [154]:
accuracy

0.779

In this problem, we use the softmax cost as the optimizer. 
We also use two ways to get the accuracy.
Now we validate the accuracy, is 0.779, which is around 0.75 but better.